FIT5196 Assessment 1 
Student Name: VADDE PAVAN KUMAR
Student ID: 29772990
Date: 14-08-2019

Version: 3.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

In [11]:
# importing the packages 
import re  # Regular expression 
import json # converting to the JSON 

# ****************extracting the units information ************************

unitguide = []
cleanr = re.compile('<.*?>') # regex used to clean up all the HTML tag lines inside the line 
count = 1
with open("29772990.txt") as fp:  # opening the file 
    line = fp.readline() #reading the file line by line 
    
    dic = {} # creating the dictionary to store all the units information 
    while line:
        # for getting out the unitcodes and unit names 
        if re.match(r'<div class="hbk-banner-box">',line):
            dic = {}
            line = fp.readline()
            line = re.sub(cleanr,'',line)  
            code = re.findall(r'[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9]',line) # regex to match unit code 
            if code:
                code = " ".join(str(x) for x in code)
                dic['unitcode'] = code # appending the unit code to the dictionary 
            name = re.findall(r'(?:[A-Z]{3}[0-9]{4} -)(.*)',line)  # regex to find the unit name 
            if name:
                name = " ".join(str(x) for x in name)
                dic['unitname'] = name
        #for getting the chief examiners         
        elif re.match(r'<p class="hbk-highlight-heading">Chief examiner\(s\)</p>',line):
            line = fp.readline()
            exam = []
            while line != "</p>\n": 
                if re.match(r'</p>',line):
                    break
                else:
                    line = re.sub(cleanr,'',line)
                    if line == "\n":
                        line = fp.readline()
                    else:
                        exam.append(line)
                        line = fp.readline()
            exam = [i.strip() for i in exam] #removing special characters 
            result = [x for x in exam if x] # creating list of chief examiners 
            for word in result:
                if word[0] == "(" or word == "Semester 1:" or word == "Semester 2:":
                    result.remove(word)
            dic["chiefexam"] = result # appending it to the dictionary 
         # for fetching the prerequisites of the unit   
        elif re.match(r'<p class="hbk-preamble-heading">Prerequisites</p>',line):
            line = fp.readline()
            line = re.sub(cleanr,'',line)
            pre = []
            while line != "</div>\n":
                if re.match(r'</div>',line) or re.match(r'<p class="hbk-preamble-heading">Co-requisites</p>',line): 
                    break
                else:
                    line = re.sub(cleanr,'',line)
                    pre.append(re.findall(r'[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9]',line))
                    line = fp.readline()
            list2 = [x for x in pre if x != []] # removing all the empty lists 
            flattened = [val for sublist in list2 for val in sublist] # joining or merging lists of lists into single list
            if len(list2) > 0: # prereqs will be appended only if there anything for the unit 
                dic["prerequisites"] = flattened
           # for fetching the corequisites of the unit   
        elif re.match(r'<p class="hbk-preamble-heading">Co-requisites</p>',line):
            line = fp.readline()
            co = []
            line = re.sub(cleanr,'',line)
            co.append(re.findall(r'[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9]',line)) # finding the unit codes excluding the text 
            list2 = [x for x in co if x != []] # removing the empty lists 
            flattened = [val for sublist in list2 for val in sublist] # merging lists of lists into single dictionary 
            if len(list2) > 0: # coreqs will be appended if only anythign they are 
                dic["corequisites"] = flattened
           #for fetching the Prohibitions of the unit 
        elif re.match(r'<p class="hbk-preamble-heading">Prohibitions</p>',line):
            line = fp.readline()
            po = []
            line = re.sub(cleanr,'',line)
            po.append(re.findall(r'[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9]',line)) # finding only the unit codes as the prohibitions 
            list2 = [x for x in po if x != []] # exclusing the empty strings 
            flattened = [val for sublist in list2 for val in sublist] # merging lsits of lists into single list 
            if len(list2) > 0:
                dic["prohibitions"] = flattened
          # for fetching the synopsis of the unit  
        elif re.match(r'<h2 class="hbk-heading">Synopsis</h2>',line):
            line = fp.readline()# reading initial couple of lines exclusing html conent 
            line = fp.readline()
            line = re.sub(cleanr,'',line) # cleaning all the html tags and assign it back to variable line
            line = line.rstrip("\n")# removing all new line characters at the end of the line 
            dic["synopsis"] = line # apeending synopis to the line 
            #print(line)
            line = fp.readline()
        # for fetching the outcomes of the data 
        elif re.match(r'<h2 class="hbk-heading">Outcomes</h2>',line):
            line = fp.readline()
            out = []
            while line != "</div>\n":
                if re.match(r'</div>',line):
                    break
                else:
                    line = re.sub(cleanr,'',line)
                    if line == "\n":
                        line = fp.readline()
                    else:
                        out.append(line)
                        line = fp.readline()
            out = [i.strip() for i in out] # removing all new line characters at the end of the line 
            out = [i.rstrip(",:;") for i in out] # removing all escape characters at the end of the line 
            dic["outcome"] = out
          # fetching the work load requirements of the unit   
        elif re.match(r'<h2 class="hbk-heading">Workload requirements</h2>',line):
            line = fp.readline()
            req = []
            while line != "</div>\n":
                if re.match(r'</div>',line):
                    break
                else:
                    line = re.sub(cleanr,'',line) # cleaning the html content from the line 
                    if line == "\n":
                        line = fp.readline()
                    else:
                        req.append(line)
                        line = fp.readline()
            req = [i.strip() for i in req] # removing all new line sequence at the end of the  line 
            req = [i.rstrip(",:;") for i in req] # removing all escape characters at the end of the line 
            dic["requirements"] = req
            #print(req)
        # if there wont be any particular requirement failed to fetch the information, then the NA as null will be appended to the dictioanry 
        elif re.match(r'<!-- /.content-inner__main --></div>',line): 
            if 'prerequisites' in dic.keys(): # checking the condition if key is in dictionary 
                pass # then pass to the next condition with out doing anything 
            else:
                dic["prerequisites"] = ["NA"]
            if 'corequisites' in dic.keys():
                pass
            else:
                dic["corequisites"] = ["NA"]
            if 'prohibitions' in dic.keys():
                pass
            else:
                dic["prohibitions"] = ["NA"]
            if 'outcome' in dic.keys():
                pass
            else:
                dic["outcome"] = ["NA"]
            if 'chiefexam' in dic.keys():
                pass
            else:
                dic["chiefexam"] = "TBA"
            if 'requirements' in dic.keys():
                pass
            else:
                dic["requirements"] = ["NA"]
            unitguide.append(dic.copy()) # after it reaches at the end of the one unit information, then everything will be appended to the dictionary 
            line =fp.readline()
        else:
            line = fp.readline() # if there is no match then continuing to the next line 

# ***************** Converting the Data to XML Format and storing in the file  *******************

f = open('29772990.xml', 'w') # creating the new file to write the data into the file
f.write('<?xml version="1.0" encoding="UTF-8"?>')
f.write('<units>') # creating the root tag XML units is the root for all the units 

for unit in unitguide :# fetching the information from the unitguide
    
    unitcode= "<unit id = " + "\"" +unit['unitcode'] + "\"" + ">" # writing the data in the specific format and appending it to the variable 
    f.write(unitcode) # writing the content to the file 
        
    f.write('<title>' + unit['unitname'] + '</title>') # title of the unit 
        
    f.write('<synopsis>' + unit['synopsis'] + '</synopsis>') # synopsis of the unit 
    
    # in this we are merging the pre and co reqs into a single pre reqs as specified 
    f.write('<pre_requistics>') # starting the prerequisites of the unit 
    for line in unit['prerequisites']: # storing prereqs in line 
        for line1 in unit['corequisites']: # stroing coreqs in line1 variable 
            if line == "NA" and line1 == "NA": # if it is NA then append it to the tag 
                f.write('<pre_requistic> ' + line + '</pre_requistic>')
            elif line == "NA" and line1 != "NA": # if prereq and no coreq append prereq
                f.write('<pre_requistic> ' + line1 + '</pre_requistic>')
            elif line != "NA" and line1 == "NA": # if prereq and no coreq append prereq
                f.write('<pre_requistic> ' + line + '</pre_requistic>')
            else:# if both has some content in them then append it to the tag 
                f.write('<pre_requistic> ' + line + '</pre_requistic>')
                f.write('<pre_requistic> ' + line1 + '</pre_requistic>')
    f.write('</pre_requistics>') # clsoing the prereqs tag 
    
    f.write('<prohibisions>') # opening tag for prohibitions 
    for line in unit['prohibitions']:
        f.write('<prohibision> ' + line + '</prohibision>')
    f.write('</prohibisions>')
    
    f.write('<requirements>') # opening tag for requirements 
    for line in unit['requirements']:
        f.write('<requirement> ' + line + '</requirement>')
    f.write('</requirements>')
    
    f.write('<outcomes>') # opening tag for outcomes 
    for line in unit['outcome']:  #Iterating through the outcomes and appending to the tag 
        f.write('<outcome> ' + line + '</outcome>')
    f.write('</outcomes>')
    
    f.write('<chief_examiners>') # opening tag for chief_examiner
    for line in unit['chiefexam']:
        f.write('<chief_examiner> ' + line + '</chief_examiner>')
    f.write('</chief_examiners>')
    
    f.write("</unit>") # close the tag for one unit 
f.write('</units>') # finnlly clsoing the units tag after fetching all the units info 
f.close() # closing the file 

# ********************* Converting Data to JSON Format and storing inthe file *******************

finall = [] # creating the finall for easy dumping of list into JSON format by predefined method with JSON package 
finals = []
guide = unitguide # getting the information from unitguide list 
for unit in guide: # iteraing through list to fetch the information 
    final = {} # creating dict to assign it all 
    final["@id"] = unit['unitcode']
    final["title"] = unit['unitname']
    final["synopsis"] = unit['synopsis']
    # creating the Dictioanry for prerequisites 
    prereq = {}
    coreq = {}
    for u in unit["prerequisites"]: # itertaing through list of prereqs 
        for u1 in unit["corequisites"]: # itertaing through list of prereqs 
            if u == "NA" and u1 == "NA": # if both are null then appending NA to list else we will be appending the rest of the content 
                final["prerequisites"] = "NA"
            elif u == "NA" and u1 != "NA":
                coreq["prerequisite"] = unit["corequisites"]
                final["prerequisites"] = coreq
            elif u != "NA" and u1 == "NA":
                prereq["prerequisite"] = unit["prerequisites"]
                final["prerequisites"] = prereq
            else:
                prereq["prerequisite"] = unit["prerequisites"] 
                final["prerequisites"] = prereq
                coreq["prerequisite"] = unit["corequisites"]
                final["prerequisites"] = coreq
    # create a dictionary for prohibitions  
    prohib = {}
    for u in unit["prohibitions"]:
        if u == "NA":
            final["prohibitions"] = "NA"
        else:
            prohib["prohibition"] = unit["prohibitions"]
            final["prohibitions"] = prohib
    # create a dictionary for requirements 
    require = {}
    for u in unit["requirements"]:
        if u == "NA":
            final["requirements"] = "NA"
        else:
            require["requirement"] = unit["requirements"]
            final["requirements"] = require
   # create a dictionary for outcomes 
    outcome = {}
    for u in unit["outcome"]:
        if u == "NA":
            final["outcomes"] = "NA"
        else:
            outcome["outcome"] = unit["outcome"]
            final["outcomes"] = outcome
    # create a dictionary for chief examiners 
    exam = {}
    for u in unit["chiefexam"]:
        if u == "TBA":
            final["chief_examiners"] = "TBA"
        else:
            exam["chief_examiner"] = unit["chiefexam"]
            final["chief_examiners"] = exam
    finall.append(final.copy())

unit ={} # creating the dictionary to store all the unit info into one dictioanry 
unit["unit"] = finall # assigning lsit to dict with the unit has key 
units = {} #creating the dictionary to store all the unit dictionary  info into one dictioanry
units["units"] = unit # assign unit to the dict with the value of units 
finals.append(units) # finally we are appending dic to lsit 
with open("29772990.json", 'w') as f: # now we are opening and creating the file to store the JSOn format
    for list in finals:
        f.write(json.dumps(list,indent=6))#write the entire list by converting it to the json format, by using json.dumps   